<a href="https://colab.research.google.com/github/lgiesen/forest-height/blob/main/21_decision_tree_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/0_load_data.ipynb"

Mounted at /content/drive


In [ ]:
# load dataset
path_train_sat = f"{root_path}train_satellite.npy"
path_train_masks = f"{root_path}train_masks.npy"

X = np.load(path_train_sat, allow_pickle=True)
y = np.load(path_train_masks, allow_pickle=True)

In [ ]:
# remove drive connection as it is no longer needed
drive.flush_and_unmount()

In [ ]:
from sklearn.model_selection import train_test_split
# split dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

## Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# instantiate the model
# TODO: Try out different values for min_samples_split
# corresponds to "m" in the slides
min_samples_split = 2

# instantiate model and fit it!
model = DecisionTreeRegressor(max_depth=5, min_samples_split=min_samples_split)

In [ ]:
def reshape_2D(var):
  if len(var.shape) == 4:
    nsamples, color_channel, nx, ny = var.shape
  else:
    nsamples, nx, ny = var.shape
    color_channel = 1
  return var.reshape((nsamples, color_channel*nx*ny))

In [ ]:
# reshape data so that DecisionTreeRegressor can work with the data
X_train_2D = reshape_2D(X_train).tolist()
y_train_2D = reshape_2D(y_train).tolist()

> Try out df as a data format. Previously tried out: ndarray, list

> Use k-fold cross-validation for hyperparameter tuning of `max_depth`

In [ ]:
del X, y

In [ ]:
%%time
# train the model
model.fit(X_train_2D, y_train_2D)
#model_2.fit(X_train_2D, y_train_2D)

In [ ]:
from sklearn.model_selection import GridSearchCV
# k-fold cross-validation for hyperparameter tuning of max_depth
gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 11),
                                'min_samples_split': range(10, 60, 10)},
                  cv=5,r
                  n_jobs=1,
                  scoring='neg_mean_squared_error')

gs.fit(X_train, y_train) # X_train_2D, y_train_2D

print(gs.best_params_)
print(-gs.best_score_)

In [ ]:
# Predict
# X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
X_test_2D = reshape_2D(X_test).tolist()
y_test_2D = reshape_2D(y_test).tolist()
preds = model.predict(X_test_2D)
#preds_model_2 = model_2.predict(y_test_2D)

In [ ]:
model.summary()

In [ ]:
# save the model to disk
import pickle
path_reg_model = root_path + 'DecisionTreeRegressor.sav'
pickle.dump(model, open(path_reg_model, 'wb'))
# loaded_model = pickle.load(open(path_reg_model, 'rb'))

In [ ]:
# save the model to disk
import pickle
from sklearn.externals import joblib
path_reg_model = root_path +  "DecisionTreeRegressor.joblib"
joblib.dump(model, path_reg_model)
# loaded_model = joblib.load(path_reg_model)

In [ ]:
# Plot the results
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test_2D, y_test_2D, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test_2D, y_test_2D, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

# from the Lecture:
X_plot = np.arange(0.0, 5.0, 0.01).reshape((-1,1))
plt.figure(figsize=(10,6))
plt.scatter(X, y, c="darkorange", label="Data", s=25)
plt.plot(X_plot, preds, color="cornflowerblue", label="Model", linewidth=3)
plt.xlabel("X", fontsize=18)
plt.ylabel("Y", fontsize=18)
plt.title(f"Regression Tree (m={min_samples_split})", fontsize=16)
plt.legend()

#
sns.scatterplot(x=df['Longitude'],
                y=df['MedHouseVal'],
                label='data')

plt.plot(df['Longitude'].sort_values(),
         dtr1.predict(df['Longitude'].sort_values().to_frame()),
         color='red', label='model',
         linewidth=2)

plt.legend()

plt.savefig('model.png')

In [ ]:
# visualize tree
from sklearn.tree import export_graphviz
import graphviz

dot_data = export_graphviz(dtr1, feature_names=['Longitude'],
                           filled=True, rounded=True)

graph = graphviz.Source(dot_data)
graph.render("tree")

Visualize Decision Tree: https://towardsdatascience.com/visualizing-decision-trees-with-python-scikit-learn-graphviz-matplotlib-1c50b4aa68dc

In [ ]:
# Evaluate Decision Tree Regressor
loss, acc = model.evaluate(X_test_2D, y_test_2D, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))